# 1 model using reguralisation

Now we'll create a model using reguralisation to increase it's performance

Let's load our data and packages first

In [4]:
%pip install matplotlib
%pip install seaborn
%pip install tensorflow
%pip install --upgrade numpy
%pip install azure-identity
%pip install azure-ai
%pip install azure-ai-ml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 22.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 92.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 MB 37.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 114.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 104.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: pr

In [5]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.python.keras.callbacks import EarlyStopping

from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, AzureCliCredential
from azure.ai.ml import MLClient

2025-06-08 15:08:39.376715: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749395319.410284    3133 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749395319.419439    3133 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749395319.444548    3133 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749395319.444591    3133 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749395319.444594    3133 computation_placer.cc:177] computation placer alr

In [39]:
credential = DefaultAzureCredential()

In [45]:
import yaml

# Load config from YAML file
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

# Extract individual variables
subscription_id = config["subscription_id"]
resource_group = config["resource_group"]
workspace_name = config["workspace_name"]

print(workspace_name)
print(subscription_id)
print(resource_group)
type(subscription_id)

mlops-project-ml
88f330cf-5648-423b-aaa3-bc9705075a61
mlops-project


str

In [41]:
ml_client = MLClient(
    credential, subscription_id, resource_group, workspace_name
)

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [42]:
from azure.ai.ml.entities import Environment
import os

custom_env_name = "aml-training"


pipeline_job_project_env = Environment(
    name="training-env",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    conda_file=os.path.join("components", "training", "conda.yaml"),
)
pipeline_job_project_env = ml_client.environments.create_or_update(pipeline_job_project_env)

print(
    f"Environment with name {pipeline_job_project_env.name} is registered to workspace, the environment version is {pipeline_job_project_env.version}"
)

Invalid type NoneType for attribute 'subscription_id' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Invalid type NoneType for attribute 'subscription_id' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types


Environment with name training-env is registered to workspace, the environment version is 1


In [43]:
from azure.ai.ml import command
from azure.ai.ml import Input, Output
import os

from azure.ai.ml import command, Input, Output
from azure.ai.ml.entities import Component

# Create the component using the Component class
training_component = command(
    name="training-1",
    display_name="Training an AI model",
    description="Trains an AI model by inputting training data CSV, augmenting it, and saving a model.",
    inputs={
        "training_data": Input(type="uri_folder"),
        "epochs": Input(type="number", default=100),
        "noise_level": Input(type="number", default=0.01),
    },
    outputs={
        "model_output": Output(type="uri_file", mode="rw_mount"),
    },
    code=".",
    command="python train.py --training_data ${{inputs.training_data}} --epochs ${{inputs.epochs}} --noise_level ${{inputs.noise_level}} --output_folder ${{outputs.model_output}}",
    environment="training-env:1",
)

In [44]:
from azure.ai.ml import dsl, Input, Output

@dsl.pipeline(
    compute="project-ml-compute",
    description="training pipeline"
)


def housing_training_pipeline(
    epochs: int,
    input_version: str = "1",
    output_version: str = "1"
):
    training_job = training_component(
        training_data= Input(
            type = "uri_folder",
            path = "azureml:data_cleaned:1"
        ),
        epochs=epochs,
        noise_level=0.01
    )

    output_path = (
        f"azureml://subscriptions/{subscription_id}/"
        f"resourcegroups/{resource_group}/"
        f"workspaces/{workspace_name}/"
        f"datastores/workspaceblobstore/paths/trained_model/"
    )

    training_job.outputs.model_output = Output(
        type="uri_file",
        path=output_path,
        name="trained_model",
        mode="rw_mount"
    )

    return {
        "output_data": training_job.outputs.model_output,
    }

# Create the pipeline instance
training_pipeline = housing_training_pipeline(
    epochs=100
)

# Submit the pipeline directly
training_pipeline_job = ml_client.jobs.create_or_update(
    training_pipeline,
    experiment_name="training_pipeline",
)

print(f"Pipeline submitted: {training_pipeline_job.studio_url}")

Invalid type NoneType for attribute 'subscription_id' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Invalid type NoneType for attribute 'subscription_id' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Invalid type NoneType for attribute 'subscription_id' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Invalid type NoneType for attribute 'resource_group_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
Invalid type NoneType for attribute 'subscription_id' value. Expected one of ['bool', 'str', 'bytes', 'int

Pipeline submitted: https://ml.azure.com/runs/affable_bridge_lbpwykv6kl?wsid=/subscriptions/88f330cf-5648-423b-aaa3-bc9705075a61/resourcegroups/mlops-project/workspaces/mlops-project-ml&tid=4ded4bb1-6bff-42b3-aed7-6a36a503bf7a
